# 🔮 | Generación de precciones

## 📘 | Librerias

In [1]:
import pandas as pd
import pickle

import warnings
warnings.filterwarnings("ignore")

## 💾 | Cargar los datos

In [2]:
df_oot_ids = pd.read_csv('../data/interim/oot.csv')
df_oot_ids.head()

,nit_enmascarado,num_oblig_orig_enmascarado,num_oblig_enmascarado
0,257335,444821,635511
1,59584,350400,730364
2,397604,973821,106521
3,368086,382995,696856
4,255009,434238,645924


In [3]:
df_test_ids = pd.read_csv('../data/processed/test.csv', usecols=['nit_enmascarado', 'num_oblig_orig_enmascarado', 'num_oblig_enmascarado'])

df_test_cols = pd.read_csv('../data/interim/test.csv')
df_test_cols = df_test_cols.drop(columns=['var_rpta_alt'])

df_test = pd.concat([df_test_ids, df_test_cols], axis=1)
df_test.head()

,nit_enmascarado,num_oblig_orig_enmascarado,num_oblig_enmascarado,max_mora,vlr_obligacion,vlr_vencido,saldo_capital,endeudamiento,cant_alter_posibles,cant_gestiones,...,canal_actualizacion_PIC,canal_actualizacion_PVI,canal_actualizacion_SDI,canal_actualizacion_SIN INFORMACION,canal_actualizacion_SVP,canal_actualizacion_VTC,cli_actualizado_S,cli_actualizado_SIN INFORMACION,ajustes_banco_NO,ajustes_banco_REDIFERIDOS
0,1,975854,104488,1.724692,-0.347160,-0.089156,0.075375,-0.740556,1.375898,-0.837510,...,False,False,False,False,True,False,False,False,True,False
1,2,389854,690004,-1.012103,-0.017895,0.720256,0.538037,0.254538,-0.938308,-0.996550,...,False,False,False,False,True,False,False,False,True,False
2,5,458725,621619,0.169695,-0.464585,0.112622,-0.303575,-0.836656,1.375898,0.991453,...,False,False,False,False,True,False,False,False,True,False
3,12,623302,457040,0.667294,2.631318,2.675555,1.424109,1.002099,0.218795,0.832413,...,False,False,False,False,True,False,True,False,True,False
4,14,209309,873421,-0.670004,0.631647,1.379254,0.924983,0.480562,0.218795,-0.280869,...,False,False,False,False,True,False,True,False,True,False


In [ ]:
df_input = pd.merge(
    df_oot_ids,
    df_test,
    on=['nit_enmascarado', 'num_oblig_orig_enmascarado','num_oblig_enmascarado'],
    how='inner'
)
df_input

## 🛬 | Importar modelo y columnas selecciondas

In [ ]:
with open("../models/columnas_modelo_lr.pkl", "rb") as f:
    columnas = pickle.load(f)

with open("../models/modelo_logistic_regression.pkl", "rb") as f:
    modelo = pickle.load(f)

## ✂️ | Separar registros a predecir

In [ ]:
X_new = df_input[columnas]
X_new

## ⚙️ | Hacer predicciones

In [ ]:
df_input["probabilidad"] = modelo.predict_proba(X_new)[:, 1]
df_input["prediccion"] = modelo.predict(X_new)

df_input[["nit_enmascarado", "num_oblig_orig_enmascarado", "num_oblig_enmascarado", "probabilidad", "prediccion"]].head()

## 🛫 | Exportar datos de entrega

In [ ]:
df_submission = df_input.copy()
df_submission = df_submission[['nit_enmascarado', 'num_oblig_orig_enmascarado', 'num_oblig_enmascarado', 'prediccion']]
df_submission['ID'] = df_submission ['nit_enmascarado'].astype(str) + '#' + df_submission ['num_oblig_orig_enmascarado'].astype(str) + '#' + df_submission ['num_oblig_enmascarado'].astype(str)
df_submission.drop(columns=['nit_enmascarado', 'num_oblig_orig_enmascarado', 'num_oblig_enmascarado'], inplace=True)
df_submission.rename(columns={'prediccion': 'var_rpta_alt'}, inplace=True)
df_submission = df_submission[['ID', 'var_rpta_alt']]
df_submission

In [ ]:
df_submission_1 = df_input.copy()
df_submission_1 = df_submission_1[['nit_enmascarado', 'num_oblig_orig_enmascarado', 'num_oblig_enmascarado', 'prediccion', 'probabilidad']]
df_submission_1['ID'] = df_submission_1 ['nit_enmascarado'].astype(str) + '#' + df_submission_1 ['num_oblig_orig_enmascarado'].astype(str) + '#' + df_submission_1 ['num_oblig_enmascarado'].astype(str)
df_submission_1.drop(columns=['nit_enmascarado', 'num_oblig_orig_enmascarado', 'num_oblig_enmascarado'], inplace=True)
df_submission_1.rename(columns={'prediccion': 'var_rpta_alt', 'probabilidad' : 'Prob_uno'}, inplace=True)
df_submission_1 = df_submission_1[['ID', 'var_rpta_alt', 'Prob_uno']]
df_submission_1

In [ ]:
df_submission.to_csv('../data/interim/submission.csv', encoding='utf-8', index=False)
df_submission_1.to_csv('../data/interim/submission_1.csv', encoding='utf-8', index=False)

In [10]:
df_input = pd.merge(
    df_oot_ids,
    df_test,
    on=['nit_enmascarado', 'num_oblig_orig_enmascarado','num_oblig_enmascarado'],
    how='left'
)
df_input

,nit_enmascarado,num_oblig_orig_enmascarado,num_oblig_enmascarado,max_mora,vlr_obligacion,vlr_vencido,saldo_capital,endeudamiento,cant_alter_posibles,cant_gestiones,...,canal_actualizacion_PIC,canal_actualizacion_PVI,canal_actualizacion_SDI,canal_actualizacion_SIN INFORMACION,canal_actualizacion_SVP,canal_actualizacion_VTC,cli_actualizado_S,cli_actualizado_SIN INFORMACION,ajustes_banco_NO,ajustes_banco_REDIFERIDOS
0,257335,444821,635511,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,59584,350400,730364,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,397604,973821,106521,-0.514504,0.272614,0.575531,0.755258,0.917443,-0.938308,1.786654,...,False,False,True,False,False,False,True,False,True,False
3,397604,973821,106521,0.418494,0.285943,0.575900,0.755258,0.926238,-0.938308,-0.121829,...,False,False,True,False,False,False,True,False,True,False
4,397604,973821,106521,1.351493,0.304971,1.139969,0.755258,0.935969,-0.938308,-0.598950,...,False,False,True,False,False,False,True,False,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
143291,293824,22004,1062743,-0.638904,0.061396,0.819899,-0.288192,0.780615,0.218795,0.832413,...,False,False,False,True,False,False,False,True,True,False
143292,293824,22004,1062743,0.325195,0.077951,1.202880,-0.288192,0.784959,0.218795,2.343295,...,False,False,False,True,False,False,False,True,True,False
143293,293824,22004,1062743,1.164893,0.094434,1.453771,-0.288192,0.799705,0.218795,-0.678470,...,False,False,False,True,False,False,False,True,True,False
143294,293824,22004,1062743,2.035692,0.097273,1.589878,-0.288192,1.084661,-0.938308,1.945694,...,False,False,False,True,False,False,False,True,True,False


In [14]:
df_input = pd.merge(
    df_oot_ids,
    df_test,
    on=['nit_enmascarado', 'num_oblig_orig_enmascarado','num_oblig_enmascarado'],
    how='left'
)
df_input

,nit_enmascarado,num_oblig_orig_enmascarado,num_oblig_enmascarado,max_mora,vlr_obligacion,vlr_vencido,saldo_capital,endeudamiento,cant_alter_posibles,cant_gestiones,...,canal_actualizacion_PIC,canal_actualizacion_PVI,canal_actualizacion_SDI,canal_actualizacion_SIN INFORMACION,canal_actualizacion_SVP,canal_actualizacion_VTC,cli_actualizado_S,cli_actualizado_SIN INFORMACION,ajustes_banco_NO,ajustes_banco_REDIFERIDOS
0,257335,444821,635511,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,59584,350400,730364,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,397604,973821,106521,-0.514504,0.272614,0.575531,0.755258,0.917443,-0.938308,1.786654,...,False,False,True,False,False,False,True,False,True,False
3,397604,973821,106521,0.418494,0.285943,0.575900,0.755258,0.926238,-0.938308,-0.121829,...,False,False,True,False,False,False,True,False,True,False
4,397604,973821,106521,1.351493,0.304971,1.139969,0.755258,0.935969,-0.938308,-0.598950,...,False,False,True,False,False,False,True,False,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
143291,293824,22004,1062743,-0.638904,0.061396,0.819899,-0.288192,0.780615,0.218795,0.832413,...,False,False,False,True,False,False,False,True,True,False
143292,293824,22004,1062743,0.325195,0.077951,1.202880,-0.288192,0.784959,0.218795,2.343295,...,False,False,False,True,False,False,False,True,True,False
143293,293824,22004,1062743,1.164893,0.094434,1.453771,-0.288192,0.799705,0.218795,-0.678470,...,False,False,False,True,False,False,False,True,True,False
143294,293824,22004,1062743,2.035692,0.097273,1.589878,-0.288192,1.084661,-0.938308,1.945694,...,False,False,False,True,False,False,False,True,True,False


In [21]:
df_input_1 = df_input.copy()
df_input_1 = df_input_1.drop_duplicates(subset=['nit_enmascarado', 'num_oblig_orig_enmascarado', 'num_oblig_enmascarado'], keep='first')
df_input_1

,nit_enmascarado,num_oblig_orig_enmascarado,num_oblig_enmascarado,max_mora,vlr_obligacion,vlr_vencido,saldo_capital,endeudamiento,cant_alter_posibles,cant_gestiones,...,canal_actualizacion_PIC,canal_actualizacion_PVI,canal_actualizacion_SDI,canal_actualizacion_SIN INFORMACION,canal_actualizacion_SVP,canal_actualizacion_VTC,cli_actualizado_S,cli_actualizado_SIN INFORMACION,ajustes_banco_NO,ajustes_banco_REDIFERIDOS
0,257335,444821,635511,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,59584,350400,730364,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,397604,973821,106521,-0.514504,0.272614,0.575531,0.755258,0.917443,-0.938308,1.786654,...,False,False,True,False,False,False,True,False,True,False
5,368086,382995,696856,-1.260903,-0.422086,-0.853201,-0.101881,-1.039909,0.218795,-0.201349,...,False,False,False,False,False,False,True,False,True,False
6,255009,434238,645924,0.262995,-0.551790,-1.108180,-1.193168,-1.582202,1.375898,0.116731,...,False,False,False,True,False,False,False,True,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
143285,316499,14503,1074624,-0.670004,-0.461175,-0.187113,-0.266067,1.276969,0.218795,0.434812,...,False,False,False,False,False,False,False,False,True,False
143288,254311,30479,1013957,-0.638904,-0.135452,1.363481,-0.173136,1.004615,0.218795,0.434812,...,False,False,False,False,False,False,True,False,True,False
143290,483193,13732,1006779,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
143291,293824,22004,1062743,-0.638904,0.061396,0.819899,-0.288192,0.780615,0.218795,0.832413,...,False,False,False,True,False,False,False,True,True,False


In [22]:
X_new_1= df_input_1[columnas]
X_new_1

,pago_mes,endeudamiento,promesas_cumplidas,tot_patrimonio,tot_activos,pago_cuota,total_ing,pago_total,rpc
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,-0.526856,0.917443,-0.321566,0.412000,0.228733,-0.775524,0.304001,-0.774785,0.267452
5,1.898052,-1.039909,-0.321566,-0.510637,-0.448283,1.289450,-0.285182,1.290680,0.267452
6,-0.526856,-1.582202,-0.321566,-0.323312,-0.301807,-0.775524,-0.210323,-0.774785,0.267452
...,...,...,...,...,...,...,...,...,...
143285,-0.526856,1.276969,-0.321566,1.324158,2.154067,-0.775524,8.613213,-0.774785,0.267452
143288,-0.526856,1.004615,-0.321566,1.170249,1.112728,-0.775524,0.131907,-0.774785,0.267452
143290,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
143291,-0.526856,0.780615,-0.321566,-0.323312,-0.301807,-0.775524,-0.210323,-0.774785,0.267452


In [23]:
X_new_1.fillna(0, inplace=True)
X_new_1

,pago_mes,endeudamiento,promesas_cumplidas,tot_patrimonio,tot_activos,pago_cuota,total_ing,pago_total,rpc
0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,-0.526856,0.917443,-0.321566,0.412000,0.228733,-0.775524,0.304001,-0.774785,0.267452
5,1.898052,-1.039909,-0.321566,-0.510637,-0.448283,1.289450,-0.285182,1.290680,0.267452
6,-0.526856,-1.582202,-0.321566,-0.323312,-0.301807,-0.775524,-0.210323,-0.774785,0.267452
...,...,...,...,...,...,...,...,...,...
143285,-0.526856,1.276969,-0.321566,1.324158,2.154067,-0.775524,8.613213,-0.774785,0.267452
143288,-0.526856,1.004615,-0.321566,1.170249,1.112728,-0.775524,0.131907,-0.774785,0.267452
143290,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
143291,-0.526856,0.780615,-0.321566,-0.323312,-0.301807,-0.775524,-0.210323,-0.774785,0.267452


In [24]:
df_input_1["probabilidad"] = modelo.predict_proba(X_new_1)[:, 1]
df_input_1["prediccion"] = modelo.predict(X_new_1)

df_input_1[["nit_enmascarado", "num_oblig_orig_enmascarado", "num_oblig_enmascarado", "probabilidad", "prediccion"]].head()

,nit_enmascarado,num_oblig_orig_enmascarado,num_oblig_enmascarado,probabilidad,prediccion
0,257335,444821,635511,0.636272,1
1,59584,350400,730364,0.636272,1
2,397604,973821,106521,0.323728,0
5,368086,382995,696856,0.855244,1
6,255009,434238,645924,0.208614,0


In [25]:
df_submission = df_input_1.copy()
df_submission = df_submission[['nit_enmascarado', 'num_oblig_orig_enmascarado', 'num_oblig_enmascarado', 'prediccion']]
df_submission['ID'] = df_submission ['nit_enmascarado'].astype(str) + '#' + df_submission ['num_oblig_orig_enmascarado'].astype(str) + '#' + df_submission ['num_oblig_enmascarado'].astype(str)
df_submission.drop(columns=['nit_enmascarado', 'num_oblig_orig_enmascarado', 'num_oblig_enmascarado'], inplace=True)
df_submission.rename(columns={'prediccion': 'var_rpta_alt'}, inplace=True)
df_submission = df_submission[['ID', 'var_rpta_alt']]
df_submission

,ID,var_rpta_alt
0,257335#444821#635511,1
1,59584#350400#730364,1
2,397604#973821#106521,0
5,368086#382995#696856,1
6,255009#434238#645924,0
...,...,...
143285,316499#14503#1074624,0
143288,254311#30479#1013957,0
143290,483193#13732#1006779,1
143291,293824#22004#1062743,0


In [26]:
df_submission.to_csv('../data/interim/submission_2.csv', encoding='utf-8', index=False)
